In [11]:
import pandas as pd
import numpy as np

In [ ]:
def check_transaction_violation(tx: dict, history_df: pd.DataFrame) -> dict:
    triggered_rules = []

    # Extract values
    age = tx.get('Customer Age', 0)
    time = tx.get('Transaction Time', 0)
    hour = time // 10000
    region_cust = tx.get('Customer Region', -1)
    region_merchant = tx.get('Merchant Region', -1)
    amount = tx.get('Transaction Amount', 0)
    date = tx.get('Transaction Date')
    month = date // 100
    sub_merchant_id = tx.get('Sub-Merchant ID')

    # --- A. Age + Time ---
    if age < 8 and hour >= 22:
        triggered_rules.append("AgeTime")

    # --- B. Restricted Hours ---
    if hour >= 23 or hour < 6:
        triggered_rules.append("RestrictedHour")

    # --- C. Lunch by Financial Aid ---
    if 11 <= hour < 13:
        triggered_rules.append("LunchTimeAid")

    # --- D. Region Mismatch (excluding online merchants)
    if region_cust != region_merchant and region_merchant != 59:
        triggered_rules.append("RegionMismatch")

    # --- E. Daily Limit ---
    same_day = (history_df['Sub-Merchant ID'] == sub_merchant_id) & (history_df['Transaction Date'] == date)
    daily_total = history_df.loc[same_day, 'Transaction Amount'].sum() + amount
    if daily_total > 500_000:
        triggered_rules.append("DailyLimit")

    # --- F. Monthly Limit ---
    history_df['Transaction_Month'] = history_df['Transaction Date'] // 100
    same_month = (history_df['Sub-Merchant ID'] == sub_merchant_id) & (history_df['Transaction_Month'] == month)
    monthly_total = history_df.loc[same_month, 'Transaction Amount'].sum() + amount
    if monthly_total > 3_000_000:
        triggered_rules.append("MonthlyLimit")

    return {
        "is_anomaly": int(len(triggered_rules) > 0),
        "triggered_rules": triggered_rules,
        "daily_total": daily_total,
        "monthly_total": monthly_total
    }


In [13]:
# Prepare historical DataFrame (loaded previously)
history_df = pd.read_csv('../dataset/cleaned_BizPlay_2024_DEC.csv')

# Example new transaction
new_tx = {
    'Customer Age': 6,
    'Transaction Time': 220012,  # 11PM
    'Customer Region': 80,
    'Merchant Region': 59,
    'Transaction Amount': 49900,
    'Transaction Date': 20250101,
    'Sub-Merchant ID': 50,
}

# Check if it's anomalous
result = check_transaction_violation(new_tx, history_df)
print("🚨 Anomaly Detected:", result['is_anomaly'])
print("❌ Violated Rules:", result['triggered_rules'])
print("💰 Daily Total:", result['daily_total'])
print("📅 Monthly Total:", result['monthly_total'])


🚨 Anomaly Detected: 1
❌ Violated Rules: ['AgeTime']
💰 Daily Total: 49900
📅 Monthly Total: 49900
